In [ ]:
!pip install -upgrade yfinanace 


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -u


In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import time
from datetime import date


### Question 1: [Index] S&P 500 Stocks Added to the Index

**Which year had the highest number of additions?**

Using the list of S&P 500 companies from Wikipedia's [S&P 500 companies page](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies), download the data including the year each Security was added to the index.

Hint: you can use [pandas.read_html](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html) to scrape the data into a DataFrame.

Steps:
1. Create a DataFrame with Security tickers, names, and the year they were added.
2. Extract the year from the addition date and calculate the number of stocks added each year.
3. Which year had the highest number of additions (1957 doesn't count, as it was the year when the S&P 500 index was Date added)? Write down this year as your answer (the most recent one, if you have several records).

*Context*: 
> "Following the announcement, all four new entrants saw their stock prices rise in extended trading on Friday" - recent examples of S&P 500 additions include DASH, WSM, EXE, TKO in 2025 ([Nasdaq article](https://www.nasdaq.com/articles/sp-500-reshuffle-dash-tko-expe-wsm-join-worth-buying)).

*Additional*: How many current S&P 500 stocks have been in the index for more than 20 years? When stocks are added to the S&P 500, they usually experience a price bump as investors and index funds buy shares following the announcement.

In [3]:
s_p_500 = pd.read_html(io='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [4]:
type(s_p_500)

list

In [5]:
sp_500 = pd.DataFrame(s_p_500[0], columns=['Symbol','Security','GICS Sector','GICS Sub-Industry','Headquarters Location','Date added','CIK'])

In [6]:
sp_500.head(3)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800


In [7]:
sp_500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 27.6+ KB


In [8]:
fig = go.Figure()

# Add markers without text
fig.add_trace(go.Scatter(
    x=sp_500['Date added'],
    y=[0]*len(sp_500),
    mode='markers',  # Remove text mode
    marker=dict(
        size=15,
        color=sp_500.index,
        colorscale='viridis',
        showscale=True
    ),
    hovertemplate='<b>%{customdata}</b><br>Date added: %{x}<extra></extra>',
    customdata=sp_500['Security']
))

# Add tilted annotations
for i, row in sp_500.iterrows():
    fig.add_annotation(
        x=row['Date added'],
        y=0,
        text=row['Security'],
        showarrow=False,
        textangle=45,  # This works for annotations!
        xshift=0,
        yshift=20,
        font=dict(size=12)
    )

# Add connecting line
fig.add_trace(go.Scatter(
    x=[sp_500['Date added'].min(), sp_500['Date added'].max()],
    y=[0, 0],
    mode='lines',
    line=dict(color='gray', width=2),
    showlegend=False
))

fig.update_layout(
    title='Company Founding Timeline',
    xaxis_title='Year Founded',
    yaxis=dict(showticklabels=False, zeroline=False, showgrid=False),
    height=500,
    hovermode='closest'
)

fig.show()

In [9]:
sp_500[sp_500.loc[:,'Date added'] == '1965-03-31']

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK
133,CMI,Cummins,Industrials,Construction Machinery & Heavy Transportation ...,"Columbus, Indiana",1965-03-31,26172
166,EMR,Emerson Electric,Industrials,Electrical Components & Equipment,"Ferguson, Missouri",1965-03-31,32604


In [10]:
sp_500['year'] = pd.to_datetime(sp_500['Date added']).dt.strftime('%Y')

In [11]:
sp_500.head(2)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,year
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1957
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,2017


In [12]:
sp_500['year'].value_counts(ascending=False)

year
1957    53
2017    23
2016    23
2019    22
2008    17
2024    16
2022    16
2021    15
2023    15
2018    14
2015    14
1997    14
2012    14
2002    13
2007    12
2020    12
2009    12
1976    11
1998    11
2006    10
2011    10
2013    10
2010     9
1999     9
2000     9
2001     8
2014     8
1985     7
1994     7
1995     7
2005     7
2004     6
1989     5
2025     5
1982     5
2003     5
1984     5
1988     4
1993     3
1980     3
1992     3
1986     3
1983     3
1981     3
1996     2
1973     2
1987     2
1972     2
1969     2
1979     2
1965     2
1975     2
1970     2
1991     1
1974     1
1964     1
1978     1
Name: count, dtype: int64

In [13]:
print('2017 had the highest number of additions to the S&P 500 outside of 1957 when the index first started')

2017 had the highest number of additions to the S&P 500 outside of 1957 when the index first started


### Question 2. [Macro] Indexes YTD (as of 1 May 2025)

**How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?**

Using Yahoo Finance World Indices data, compare the year-to-date (YTD) performance (1 January-1 May 2025) of major stock market indexes for the following countries:
* United States - S&P 500 (^GSPC)
* China - Shanghai Composite (000001.SS)
* Hong Kong - HANG SENG INDEX (^HSI)	
* Australia - S&P/ASX 200 (^AXJO)
* India - Nifty 50 (^NSEI)
* Canada - S&P/TSX Composite (^GSPTSE)
* Germany - DAX (^GDAXI)
* United Kingdom - FTSE 100 (^FTSE)
* Japan - Nikkei 225 (^N225)
* Mexico - IPC Mexico (^MXX)
* Brazil - Ibovespa (^BVSP)

*Hint*: use start_date='2025-01-01' and end_date='2025-05-01' when downloading daily data in yfinance

Context: 
> [Global Valuations: Who's Cheap, Who's Not?](https://simplywall.st/article/beyond-the-us-global-markets-after-yet-another-tariff-update) article suggests "Other regions may be growing faster than the US and you need to diversify."

Reference: Yahoo Finance World Indices - https://finance.yahoo.com/world-indices/

*Additional*: How many of these indexes have better returns than the S&P 500 over 3, 5, and 10 year periods? Do you see the same trend?
Note: For simplicity, ignore currency conversion effects.)

In [14]:
start_q2 = date(year=2025, month=1, day=1)
end_q2 = date(year=2025, month=5, day=1)

In [15]:
tickers = ["^GSPC","000001.SS","^HSI","^AXJO","^NSEI","^GSPTSE","^GDAXI","^FTSE","^N225","^MXX","^BVSP"]
indexes = yf.download(tickers=tickers, start="2025-01-01", end="2025-05-02")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  11 of 11 completed


In [16]:
type(indexes)

pandas.core.frame.DataFrame

In [17]:
indexes.head(2)

Price             Close                                                    \
Ticker        000001.SS        ^AXJO     ^BVSP        ^FTSE        ^GDAXI   
Date                                                                        
2025-01-01          NaN          NaN       NaN          NaN           NaN   
2025-01-02  3262.561035  8201.200195  120125.0  8260.099609  20024.660156   

Price                                                               ...  \
Ticker            ^GSPC  ^GSPTSE          ^HSI          ^MXX ^N225  ...   
Date                                                                ...   
2025-01-01          NaN      NaN           NaN           NaN   NaN  ...   
2025-01-02  5868.549805  24898.0  19623.320312  49765.199219   NaN  ...   

Price         Volume                                                    \
Ticker         ^AXJO      ^BVSP        ^FTSE      ^GDAXI         ^GSPC   
Date                                                                     
2025-01-01       NaN        NaN          NaN         NaN           NaN   
2025-01-02  304400.0  9373600.0  422219900.0  52445600.0  3.621680e+09   

Price                                                              
Ticker          ^GSPTSE          ^HSI        ^MXX ^N225     ^NSEI  
Date                                                               
2025-01-01          NaN           NaN         NaN   NaN  154900.0  
2025-01-02  215089400.0  4.033400e+09  87535300.0   NaN  283200.0  

[2 rows x 55 columns]

In [18]:
indexes.tail(2)

Price             Close                                                    \
Ticker        000001.SS        ^AXJO     ^BVSP        ^FTSE        ^GDAXI   
Date                                                                        
2025-04-30  3279.031006  8126.200195  135067.0  8494.900391  22496.980469   
2025-05-01          NaN  8145.600098       NaN  8496.799805           NaN   

Price                                                             \
Ticker            ^GSPC       ^GSPTSE          ^HSI         ^MXX   
Date                                                               
2025-04-30  5569.060059  24841.699219  22119.410156  56259.28125   
2025-05-01  5604.140137  24795.599609           NaN          NaN   

Price                     ...    Volume                                        \
Ticker             ^N225  ...     ^AXJO       ^BVSP         ^FTSE      ^GDAXI   
Date                      ...                                                   
2025-04-30  36045.378906  ...  889500.0  15452400.0  1.589592e+09  99189800.0   
2025-05-01  36452.300781  ...  676700.0         NaN  5.275688e+08         NaN   

Price                                                                          \
Ticker             ^GSPC      ^GSPTSE          ^HSI         ^MXX        ^N225   
Date                                                                            
2025-04-30  5.449490e+09  271264200.0  4.682300e+09  226957400.0  167100000.0   
2025-05-01  4.935270e+09  213319000.0           NaN          NaN  131600000.0   

Price                 
Ticker         ^NSEI  
Date                  
2025-04-30  424500.0  
2025-05-01       NaN  

[2 rows x 55 columns]

In [19]:
indexes.columns

MultiIndex([( 'Close', '000001.SS'),
            ( 'Close',     '^AXJO'),
            ( 'Close',     '^BVSP'),
            ( 'Close',     '^FTSE'),
            ( 'Close',    '^GDAXI'),
            ( 'Close',     '^GSPC'),
            ( 'Close',   '^GSPTSE'),
            ( 'Close',      '^HSI'),
            ( 'Close',      '^MXX'),
            ( 'Close',     '^N225'),
            ( 'Close',     '^NSEI'),
            (  'High', '000001.SS'),
            (  'High',     '^AXJO'),
            (  'High',     '^BVSP'),
            (  'High',     '^FTSE'),
            (  'High',    '^GDAXI'),
            (  'High',     '^GSPC'),
            (  'High',   '^GSPTSE'),
            (  'High',      '^HSI'),
            (  'High',      '^MXX'),
            (  'High',     '^N225'),
            (  'High',     '^NSEI'),
            (   'Low', '000001.SS'),
            (   'Low',     '^AXJO'),
            (   'Low',     '^BVSP'),
            (   'Low',     '^FTSE'),
            (   'Low',    '^GDAXI'),
 

In [20]:
indexes["Close"].head()

Ticker,000001.SS,^AXJO,^BVSP,^FTSE,^GDAXI,^GSPC,^GSPTSE,^HSI,^MXX,^N225,^NSEI
Date,,,,,,,,,,,
2025-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23742.900391
2025-01-02,3262.561035,8201.200195,120125.0,8260.099609,20024.660156,5868.549805,24898.000000,19623.320312,49765.199219,NaN,24188.650391
2025-01-03,3211.429932,8250.500000,118533.0,8224.000000,19906.080078,5942.470215,25073.500000,19760.269531,48957.238281,NaN,24004.750000
2025-01-06,3206.923096,8288.500000,120022.0,8249.700195,20216.189453,5975.379883,24999.800781,19688.289062,49493.558594,39307.050781,23616.050781
2025-01-07,3229.644043,8285.099609,121163.0,8245.299805,20340.570312,5909.029785,24929.900391,19447.580078,50085.500000,40083.300781,23707.900391


In [21]:
indexes["Close"].tail()

Ticker,000001.SS,^AXJO,^BVSP,^FTSE,^GDAXI,^GSPC,^GSPTSE,^HSI,^MXX,^N225,^NSEI
Date,,,,,,,,,,,
2025-04-25,3295.060059,NaN,134739.0,8415.299805,22242.449219,5525.209961,24710.500000,21980.740234,56720.121094,35705.738281,24039.349609
2025-04-28,3288.415039,7997.100098,135016.0,8417.299805,22271.669922,5528.750000,24798.599609,21971.960938,56980.128906,35839.988281,24328.500000
2025-04-29,3286.655029,8070.600098,135093.0,8463.500000,22425.830078,5560.830078,24874.500000,22008.109375,55613.429688,NaN,24335.949219
2025-04-30,3279.031006,8126.200195,135067.0,8494.900391,22496.980469,5569.060059,24841.699219,22119.410156,56259.281250,36045.378906,24334.199219
2025-05-01,NaN,8145.600098,NaN,8496.799805,NaN,5604.140137,24795.599609,NaN,NaN,36452.300781,NaN


In [22]:
close_indexes = indexes["Close"]

In [23]:
close_indexes

Ticker,000001.SS,^AXJO,^BVSP,^FTSE,^GDAXI,^GSPC,^GSPTSE,^HSI,^MXX,^N225,^NSEI
Date,,,,,,,,,,,
2025-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23742.900391
2025-01-02,3262.561035,8201.200195,120125.0,8260.099609,20024.660156,5868.549805,24898.000000,19623.320312,49765.199219,NaN,24188.650391
2025-01-03,3211.429932,8250.500000,118533.0,8224.000000,19906.080078,5942.470215,25073.500000,19760.269531,48957.238281,NaN,24004.750000
2025-01-06,3206.923096,8288.500000,120022.0,8249.700195,20216.189453,5975.379883,24999.800781,19688.289062,49493.558594,39307.050781,23616.050781
2025-01-07,3229.644043,8285.099609,121163.0,8245.299805,20340.570312,5909.029785,24929.900391,19447.580078,50085.500000,40083.300781,23707.900391
...,...,...,...,...,...,...,...,...,...,...,...
2025-04-25,3295.060059,NaN,134739.0,8415.299805,22242.449219,5525.209961,24710.500000,21980.740234,56720.121094,35705.738281,24039.349609
2025-04-28,3288.415039,7997.100098,135016.0,8417.299805,22271.669922,5528.750000,24798.599609,21971.960938,56980.128906,35839.988281,24328.500000
2025-04-29,3286.655029,8070.600098,135093.0,8463.500000,22425.830078,5560.830078,24874.500000,22008.109375,55613.429688,NaN,24335.949219


In [24]:
jan_may = {}

for ticker in tickers:
    start_price = close_indexes[ticker].dropna().iloc[0]
    end_price = close_indexes[ticker].dropna().iloc[-1]
    ytd = ((end_price / start_price)-1)*100
    jan_may[ticker] = np.round(ytd,2)

In [25]:
jan_may

{'^GSPC': np.float64(-4.51),
 '000001.SS': np.float64(0.5),
 '^HSI': np.float64(12.72),
 '^AXJO': np.float64(-0.68),
 '^NSEI': np.float64(2.49),
 '^GSPTSE': np.float64(-0.41),
 '^GDAXI': np.float64(12.35),
 '^FTSE': np.float64(2.87),
 '^N225': np.float64(-7.26),
 '^MXX': np.float64(13.05),
 '^BVSP': np.float64(12.44)}

In [26]:
print(f"There are 9 indexes out of 10 that have better year-to-date returns than the US (S&P 500) as of May 1, 2025")

There are 9 indexes out of 10 that have better year-to-date returns than the US (S&P 500) as of May 1, 2025


### Question 3. [Index] S&P 500 Market Corrections Analysis


**Calculate the median duration (in days) of significant market corrections in the S&P 500 index.**

For this task, define a correction as an event when a stock index goes down by **more than 5%** from the closest all-time high maximum.

Steps:
1. Download S&P 500 historical data (1950-present) using yfinance
2. Identify all-time high points (where price exceeds all previous prices)
3. For each pair of consecutive all-time highs, find the minimum price in between
4. Calculate drawdown percentages: (high - low) / high × 100
5. Filter for corrections with at least 5% drawdown
6. Calculate the duration in days for each correction period
7. Determine the 25th, 50th (median), and 75th percentiles for correction durations

*Context:* 
> * Investors often wonder about the typical length of market corrections when deciding "when to buy the dip" ([Reddit discussion](https://www.reddit.com/r/investing/comments/1jrqnte/when_are_you_buying_the_dip/?rdt=64135)).

> * [A Wealth of Common Sense - How Often Should You Expect a Stock Market Correction?](https://awealthofcommonsense.com/2022/01/how-often-should-you-expect-a-stock-market-correction/)

*Hint (use this data to compare with your results)*: Here is the list of top 10 largest corrections by drawdown:
* 2007-10-09 to 2009-03-09: 56.8% drawdown over 517 days
* 2000-03-24 to 2002-10-09: 49.1% drawdown over 929 days
* 1973-01-11 to 1974-10-03: 48.2% drawdown over 630 days
* 1968-11-29 to 1970-05-26: 36.1% drawdown over 543 days
* 2020-02-19 to 2020-03-23: 33.9% drawdown over 33 days
* 1987-08-25 to 1987-12-04: 33.5% drawdown over 101 days
* 1961-12-12 to 1962-06-26: 28.0% drawdown over 196 days
* 1980-11-28 to 1982-08-12: 27.1% drawdown over 622 days
* 2022-01-03 to 2022-10-12: 25.4% drawdown over 282 days
* 1966-02-09 to 1966-10-07: 22.2% drawdown over 240 days

In [27]:
start_q3 = date(year=1950, month=1, day=1)

In [28]:
sp_obj = yf.Ticker("^GSPC")
sp_500_q3 = sp_obj.history(start=start_q3)

In [29]:
sp_500_q3.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1950-01-03 00:00:00-05:00,16.66,16.66,16.66,16.66,1260000,0.0,0.0
1950-01-04 00:00:00-05:00,16.85,16.85,16.85,16.85,1890000,0.0,0.0
1950-01-05 00:00:00-05:00,16.93,16.93,16.93,16.93,2550000,0.0,0.0
1950-01-06 00:00:00-05:00,16.98,16.98,16.98,16.98,2010000,0.0,0.0
1950-01-09 00:00:00-05:00,17.08,17.08,17.08,17.08,2520000,0.0,0.0


In [30]:
len(sp_500_q3['Close'])

18977

#change the data type of the index
sp_500_q3.index = pd.to_datetime(sp_500_q3.index, format="%Y-%m-%d")

In [31]:
def rolling_ath(df: pd.DataFrame, price_column: str = 'Close', date_column: str = None) -> pd.DataFrame:
    """
    Enhanced pandas function to identify new ATH points with additional metrics.
    
    Args:
        df: DataFrame with stock data
        price_column: Column name for prices
        date_column: Column name for dates (None if date is the index)
    
    Returns:
        DataFrame with rolling ATH and new ATH indicators
    """
    result_df = df.copy()
    
    # Calculate rolling maximum (ATH at each point)
    result_df['rolling_ath'] = df[price_column].cummax()
    
    # Flag new ATH points
    result_df['is_new_ath'] = df[price_column] == result_df['rolling_ath']
    
    # Remove consecutive duplicate ATH flags (keep only the first occurrence)
    result_df['is_new_ath'] = result_df['is_new_ath'] & (result_df['rolling_ath'] != result_df['rolling_ath'].shift(1))
    
    # Calculate days/periods since last ATH
    result_df['periods_since_ath'] = 0
    
    # If date is in index and it's a DatetimeIndex, calculate actual days
    if date_column is None and isinstance(result_df.index, pd.DatetimeIndex):
        ath_mask = result_df['is_new_ath']
        ath_dates = result_df.index[ath_mask]
        
        for i, current_date in enumerate(result_df.index):
            if result_df.iloc[i]['is_new_ath']:
                result_df.iloc[i, result_df.columns.get_loc('periods_since_ath')] = 0
            else:
                # Find the most recent ATH date before current date
                recent_ath_dates = ath_dates[ath_dates < current_date]
                if len(recent_ath_dates) > 0:
                    last_ath_date = recent_ath_dates[-1]
                    days_diff = (current_date - last_ath_date).days
                    result_df.iloc[i, result_df.columns.get_loc('periods_since_ath')] = days_diff
                else:
                    # No previous ATH found, use position-based counting
                    result_df.iloc[i, result_df.columns.get_loc('periods_since_ath')] = i
    
    # If date is a column or index is not datetime, use position-based counting
    else:
        for i in range(len(result_df)):
            if result_df.iloc[i]['is_new_ath']:
                result_df.iloc[i, result_df.columns.get_loc('periods_since_ath')] = 0
            elif i > 0:
                result_df.iloc[i, result_df.columns.get_loc('periods_since_ath')] = \
                    result_df.iloc[i-1]['periods_since_ath'] + 1
    
    return result_df


In [32]:
rolling_ath(sp_500_q3).head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,rolling_ath,is_new_ath,periods_since_ath
Date,,,,,,,,,,
1950-01-03 00:00:00-05:00,16.66,16.66,16.66,16.66,1260000,0.0,0.0,16.66,True,0
1950-01-04 00:00:00-05:00,16.85,16.85,16.85,16.85,1890000,0.0,0.0,16.85,True,0
1950-01-05 00:00:00-05:00,16.93,16.93,16.93,16.93,2550000,0.0,0.0,16.93,True,0
1950-01-06 00:00:00-05:00,16.98,16.98,16.98,16.98,2010000,0.0,0.0,16.98,True,0
1950-01-09 00:00:00-05:00,17.08,17.08,17.08,17.08,2520000,0.0,0.0,17.08,True,0


In [33]:
def find_troughs_between_aths(df: pd.DataFrame, price_column: str = 'Close') -> pd.DataFrame:
    """
    Find the lowest points (troughs) between each all-time high.
    
    Args:
        df: DataFrame with stock data (date as index or column)
        price_column: Column name for prices
    
    Returns:
        DataFrame with ATH points and corresponding troughs between them
    """
    # First get the ATH analysis
    ath_df = rolling_ath(df, price_column)
    
    # Get indices where new ATHs occur
    ath_indices = ath_df[ath_df['is_new_ath']].index.tolist()
    
    if len(ath_indices) <= 1:
        return pd.DataFrame()  # Need at least 2 ATHs to find troughs between them
    
    trough_data = []
    
    for i in range(len(ath_indices) - 1):
        # Define the period between current ATH and next ATH
        start_idx = ath_indices[i]
        end_idx = ath_indices[i + 1]
        
        # Get the slice of data between these ATHs (exclusive of endpoints)
        if isinstance(df.index, pd.RangeIndex):
            # For integer index
            start_pos = df.index.get_loc(start_idx)
            end_pos = df.index.get_loc(end_idx)
            between_slice = df.iloc[start_pos + 1:end_pos]
        else:
            # For other index types (like DatetimeIndex)
            between_slice = df.loc[start_idx:end_idx].iloc[1:-1]
        
        if len(between_slice) > 0:
            # Find the minimum price in this period
            min_idx = between_slice[price_column].idxmin()
            min_price = between_slice.loc[min_idx, price_column]
            
            # Get corresponding ATH values
            ath_price_start = df.loc[start_idx, price_column]
            ath_price_end = df.loc[end_idx, price_column]
            
            # Calculate drawdown percentage from the previous ATH
            drawdown_pct = ((min_price - ath_price_start) / ath_price_start) * 100
            
            trough_data.append({
                'ath_start_date': start_idx,
                'ath_start_price': ath_price_start,
                'trough_date': min_idx,
                'trough_price': min_price,
                'ath_end_date': end_idx,
                'ath_end_price': ath_price_end,
                'drawdown_pct': drawdown_pct,
                'recovery_pct': ((ath_price_end - min_price) / min_price) * 100,
                'duration': min_idx - start_idx
            })
    
    return pd.DataFrame(trough_data)


In [34]:
def add_trough_flags_to_df(df: pd.DataFrame, price_column: str = 'close') -> pd.DataFrame:
    """
    Add trough identification flags to the original DataFrame.
    
    Args:
        df: DataFrame with stock data
        price_column: Column name for prices
    
    Returns:
        DataFrame with additional columns for trough analysis
    """
    result_df = rolling_ath(df, price_column)
    
    # Initialize trough flags
    result_df['is_trough'] = False
    result_df['trough_between_aths'] = False
    result_df['drawdown_from_ath_pct'] = 0.0
    
    # Get trough data
    troughs_df = find_troughs_between_aths(df, price_column)
    
    if not troughs_df.empty:
        # Mark trough points
        trough_indices = troughs_df['trough_date'].tolist()
        result_df.loc[trough_indices, 'is_trough'] = True
        result_df.loc[trough_indices, 'trough_between_aths'] = True
        
        # Calculate drawdown from most recent ATH for all points
        ath_indices = result_df[result_df['is_new_ath']].index.tolist()
        
        for i, idx in enumerate(result_df.index):
            current_price = result_df.loc[idx, price_column]
            
            # Find the most recent ATH before or at this point
            recent_aths = [ath_idx for ath_idx in ath_indices if ath_idx <= idx]
            if recent_aths:
                most_recent_ath_idx = recent_aths[-1]
                ath_price = result_df.loc[most_recent_ath_idx, price_column]
                drawdown_pct = ((current_price - ath_price) / ath_price) * 100
                result_df.loc[idx, 'drawdown_from_ath_pct'] = drawdown_pct
    
    return result_df

In [35]:
q3_df = find_troughs_between_aths(sp_500_q3)

In [36]:
q3_df.head(8)

,ath_start_date,ath_start_price,trough_date,trough_price,ath_end_date,ath_end_price,drawdown_pct,recovery_pct,duration
0,1950-01-09 00:00:00-05:00,17.080000,1950-01-10 00:00:00-05:00,17.030001,1950-01-11 00:00:00-05:00,17.090000,-0.292736,0.352316,1 days
1,1950-01-11 00:00:00-05:00,17.090000,1950-01-13 00:00:00-05:00,16.670000,1950-02-02 00:00:00-05:00,17.230000,-2.457578,3.359325,2 days
2,1950-02-06 00:00:00-05:00,17.320000,1950-02-16 00:00:00-05:00,16.990000,1950-03-15 00:00:00-05:00,17.450001,-1.905311,2.707481,10 days
3,1950-03-16 00:00:00-05:00,17.490000,1950-03-20 00:00:00-05:00,17.440001,1950-03-22 00:00:00-05:00,17.549999,-0.285873,0.630726,4 days
4,1950-03-23 00:00:00-05:00,17.559999,1950-03-31 00:00:00-05:00,17.290001,1950-04-05 00:00:00-05:00,17.629999,-1.537577,1.966444,8 days
5,1950-04-10 00:00:00-05:00,17.850000,1950-04-11 00:00:00-05:00,17.750000,1950-04-12 00:00:00-05:00,17.940001,-0.560226,1.070426,1 days
6,1950-04-13 00:00:00-05:00,17.980000,1950-04-17 00:00:00-05:00,17.879999,1950-04-18 00:00:00-05:00,18.030001,-0.556176,0.838935,4 days
7,1950-04-19 00:00:00-05:00,18.049999,1950-04-26 00:00:00-05:00,17.760000,1950-05-01 00:00:00-04:00,18.219999,-1.606643,2.590085,7 days


In [37]:
q3_df = q3_df[q3_df['drawdown_pct'] < -5]

In [38]:
q3_df['duration'].describe()

count                             71
mean     113 days 02:38:01.690140846
std      179 days 02:11:26.722984390
min                  7 days 00:00:00
25%                 21 days 12:00:00
50%                 39 days 00:00:00
75%                 89 days 00:00:00
max                928 days 23:00:00
Name: duration, dtype: object

In [39]:
percentiles = np.percentile(q3_df['duration'], [25, 50, 75])

In [40]:
np.percentile(q3_df['duration'], [25, 50, 75])

array([1857600000000000, 3369600000000000, 7689600000000000],
      dtype='timedelta64[ns]')

In [41]:
print(f"The median duration of significant market corrections in the S&P 500 index is: {str(q3_df['duration'].describe().iloc[5]).replace(' 00:00:00', '')}")

The median duration of significant market corrections in the S&P 500 index is: 39 days


### Question 4.  [Stocks] Earnings Surprise Analysis for Amazon (AMZN)


**Calculate the median 2-day percentage change in stock prices following positive earnings surprises days.**

Steps:
1. Load earnings data from CSV ([ha1_Amazon.csv](ha1_Amazon.csv)) containing earnings dates, EPS estimates, and actual EPS
2. Download complete historical price data using yfinance
3. Calculate 2-day percentage changes for all historical dates: for each sequence of 3 consecutive trading days (Day 1, Day 2, Day 3), compute the return as Close_Day3 / Close_Day1 - 1. (Assume Day 2 may correspond to the earnings announcement.)
4. Identify positive earnings surprises (where "actual EPS > estimated EPS" OR "Surprise (%)>0")
5. Calculate 2-day percentage changes following positive earnings surprises
6. Compare the median 2-day percentage change for positive surprises vs. all historical dates

Context: Earnings announcements, especially when they exceed analyst expectations, can significantly impact stock prices in the short term.

Reference: Yahoo Finance earnings calendar - https://finance.yahoo.com/calendar/earnings?symbol=AMZN

*Additional*: Is there a correlation between the magnitude of the earnings surprise and the stock price reaction? Does the market react differently to earnings surprises during bull vs. bear markets?)

In [42]:
amazon = pd.read_csv('ha1_Amazon.csv', sep=";")

In [43]:
amazon.head(2)

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
0,AMZN,Amazon.com Inc,"April 29, 2026 at 6 AM EDT",-,-,-
1,AMZN,Amazon.com Inc,"February 4, 2026 at 4 PM EST",-,-,-


In [44]:
amazon.describe()

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
count,116,116,116,116,116,116
unique,1,2,116,38,39,113
top,AMZN,"Amazon.com, Inc.","April 29, 2026 at 6 AM EDT",-,0.01,-
freq,116,112,1,28,24,4


In [45]:
#Replace - with np.nan and drop the null values
amazon = amazon.replace("-", np.nan).dropna()

In [46]:
#remove ??? with 0
amazon['EPS Estimate'] = amazon['EPS Estimate'].str.replace('?', '0', regex=False).astype(float)
amazon['Reported EPS'] = amazon['Reported EPS'].str.replace('?', '0', regex=False).astype(float)

In [47]:
#remove + from the Surprise (%) column
amazon['Surprise (%)'] = amazon['Surprise (%)'].str.replace(r'^\+', '', regex=True).astype(float)

In [48]:
amazon.head(2)

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
4,AMZN,"Amazon.com, Inc.","May 1, 2025 at 4 PM EDT",0.36,0.59,16.74
5,AMZN,"Amazon.com, Inc.","February 6, 2025 at 4 PM EST",0.49,0.86,24.47


In [49]:
amazon.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88 entries, 4 to 108
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Symbol         88 non-null     object 
 1   Company        88 non-null     object 
 2   Earnings Date  88 non-null     object 
 3   EPS Estimate   88 non-null     float64
 4   Reported EPS   88 non-null     float64
 5   Surprise (%)   88 non-null     float64
dtypes: float64(3), object(3)
memory usage: 4.8+ KB


In [50]:
amzn = yf.download(tickers='amzn')

[*********************100%***********************]  1 of 1 completed


In [51]:
amzn.tail()

Price,Close,High,Low,Open,Volume
Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
Date,,,,,
2025-05-30,205.009995,205.990005,201.699997,204.839996,51679400
2025-06-02,206.649994,207.000000,202.679993,204.979996,29113300
2025-06-03,205.710007,208.949997,205.029999,207.110001,33139100
2025-06-04,207.229996,208.179993,205.179993,206.550003,29866400
2025-06-05,211.285004,211.630005,208.050003,209.500000,18355482


In [52]:
amzn_2day_change = amzn.copy()

amzn_2day_change['2day_change_percent'] = amzn_2day_change['Close'].shift(-3) / amzn_2day_change['Close'] - 1

In [53]:
amzn_2day_change.columns = amzn_2day_change.columns.droplevel(1)

In [54]:
amzn_2day_change.head()

Price,Close,High,Low,Open,Volume,2day_change_percent
Date,,,,,,
1997-05-15,0.097917,0.125000,0.096354,0.121875,1443120000,-0.164895
1997-05-16,0.086458,0.098958,0.085417,0.098438,294000000,-0.174697
1997-05-19,0.085417,0.088542,0.081250,0.088021,122136000,-0.182926
1997-05-20,0.081771,0.087500,0.081771,0.086458,109344000,-0.082804
1997-05-21,0.071354,0.082292,0.068750,0.081771,377064000,0.109496


In [55]:
for col in amzn_2day_change.columns.values:
    print(col)

Close
High
Low
Open
Volume
2day_change_percent


In [56]:
#create a dataframe with only the positive earning surprises

positive_earnings = amazon[(amazon['Reported EPS'] > amazon['EPS Estimate']) ]

In [57]:
#create a new column for the date with a new format
earn_date = positive_earnings["Earnings Date"].str.replace(r'\s+[A-Z]{3,4}$', '', regex=True)
positive_earnings.index = pd.to_datetime(earn_date, format='%B %d, %Y at %I %p').dt.date

In [58]:
positive_earnings.head(7)

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
Earnings Date,,,,,,
2025-05-01,AMZN,"Amazon.com, Inc.","May 1, 2025 at 4 PM EDT",0.36,0.59,16.74
2025-02-06,AMZN,"Amazon.com, Inc.","February 6, 2025 at 4 PM EST",0.49,0.86,24.47
2024-10-31,AMZN,"Amazon.com, Inc.","October 31, 2024 at 4 PM EDT",0.14,0.43,25.17
2024-04-30,AMZN,"Amazon.com, Inc.","April 30, 2024 at 4 PM EDT",0.83,0.98,17.91
2024-02-01,AMZN,"Amazon.com, Inc.","February 1, 2024 at 4 PM EST",0.80,1.00,24.55
2023-10-26,AMZN,"Amazon.com, Inc.","October 26, 2023 at 4 PM EDT",0.58,0.94,60.85
2023-08-03,AMZN,"Amazon.com, Inc.","August 3, 2023 at 4 PM EDT",0.35,0.65,85.73


In [59]:
positive_earnings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 2025-05-01 to 2006-02-02
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Symbol         36 non-null     object 
 1   Company        36 non-null     object 
 2   Earnings Date  36 non-null     object 
 3   EPS Estimate   36 non-null     float64
 4   Reported EPS   36 non-null     float64
 5   Surprise (%)   36 non-null     float64
dtypes: float64(3), object(3)
memory usage: 2.0+ KB


In [60]:
positive_earnings[positive_earnings["EPS Estimate"] > positive_earnings["Reported EPS"]]

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
Earnings Date,,,,,,


In [61]:
positive_earnings[positive_earnings["Surprise (%)"] < 0]

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
Earnings Date,,,,,,


In [62]:
# Join the data frame with only the positive earnings surpises to the stock price dataframe
q4_df = amzn_2day_change.merge(positive_earnings, how='left', left_index=True, right_index=True)

In [63]:
q4_df_earnings_date = q4_df[q4_df["Earnings Date"].notna()]

In [64]:
q4_df_no_earnings_date = q4_df[~q4_df["Earnings Date"].notna()]

In [65]:
q4_df.loc["08-02-2024"]

Close                  167.899994
High                   168.770004
Low                    160.550003
Open                       166.75
Volume                  141448400
2day_change_percent     -0.030554
Symbol                        NaN
Company                       NaN
Earnings Date                 NaN
EPS Estimate                  NaN
Reported EPS                  NaN
Surprise (%)                  NaN
Name: 2024-08-02 00:00:00, dtype: object

In [66]:
q4_df_earnings_date.describe()

,Close,High,Low,Open,Volume,2day_change_percent,EPS Estimate,Reported EPS,Surprise (%)
count,36.000000,36.000000,36.000000,36.000000,3.600000e+01,36.000000,36.000000,36.000000,36.000000
mean,93.781625,95.121806,92.432250,93.825097,1.712250e+08,0.025679,0.211944,0.337500,81.145278
std,67.061647,68.022979,66.365583,67.396974,9.037238e+07,0.086547,0.228020,0.320146,106.818302
min,2.137000,2.202000,1.990000,2.192000,6.089710e+07,-0.141260,-0.010000,0.010000,3.030000
25%,29.623875,30.619375,29.428374,30.116500,1.093275e+08,-0.034791,0.037500,0.050000,24.530000
50%,91.966999,92.938751,91.377750,92.424999,1.530000e+08,0.029339,0.140000,0.270000,55.365000
75%,154.265495,155.890003,151.592499,151.992496,2.002945e+08,0.084034,0.352500,0.597500,88.007500
max,238.830002,239.660004,236.009995,238.009995,4.741860e+08,0.271800,0.830000,1.000000,605.290000


In [67]:
print(f"The median 2 day price change after earnings realease:{q4_df_earnings_date[['Close','2day_change_percent']].describe().iloc[5,1] * 100}%")

The median 2 day price change after earnings realease:2.9339017113609533%


In [68]:
print(f"The median 2 day price change without earnings release:{q4_df_no_earnings_date[['Close','2day_change_percent']].describe().iloc[5,1] * 100}%")

The median 2 day price change without earnings release:0.3084479579395172%


### Question 5.  [Exploratory, optional] Brainstorm potential idea for your capstone project

**Free text answer**

Describe the capstone project you would like to pursue, considering your aspirations, ML model predictions, and prior knowledge. Even if you are unsure at this stage, try to generate an idea you would like to explore-such as a specific asset class, country, industry vertical, or investment strategy. Be as specific as possible.

*Example: I want to build a short-term prediction model for the US/India/Brazil stock markets, focusing on the largest stocks over a 30-day investment horizon. I plan to use RSI and MACD technical indicators and news coverage data to generate predictions.*


In [69]:
print(f"I am thinking about creating an algorithm for trading U.S. commodities over a two-week horizon. \nI plan to use RSI aand MACD technical indicators for tracking the price movement and bolinger bands for tracking the volitility")

I am thinking about creating an algorithm for trading U.S. commodities over a two-week horizon. 
I plan to use RSI aand MACD technical indicators for tracking the price movement and bolinger bands for tracking the volitility


### Question 6. [Exploratory, optional] Investigate new metrics

**Free text answer**

Using the data sources we have covered (or any others you find relevant), download and explore a few additional metrics or time series that could be valuable for your project. Briefly explain why you think each metric is useful. This does not need to be a comprehensive list-focus on demonstrating your ability to generate data requests based on your project description, identify and locate the necessary data, and explain how you would retrieve it using Python.

In [70]:
print(f"I think having all the tarrif rates around the world would be helpful for my analysis. Also, I would need production rates and reserves around the world. \nHowever, scrapping the web for news will be helpful for enhancing the accuracy of the predictions")

I think having all the tarrif rates around the world would be helpful for my analysis. Also, I would need production rates and reserves around the world. 
However, scrapping the web for news will be helpful for enhancing the accuracy of the predictions
